## Ejercicio / Tarea 9
#### Ariel Vallarino - 175875
---  
Aprovecha la capacidad de Dask para realizar cómputo en paralelo para ajustar un modelo para predecir la proporción de propina de un viaje. Realiza búsqueda de hiperparámetros en grid con cross validation. Puedes usar funciones de scikit learn. Recuerda usar el decorador `delayed` para ejecutar en paralelo.

* ¿Qué tan rápido es buscar en paralelo comparado con una búsqueda secuencial en python?

Haz lo mismo que arriba, pero utilizando la biblioteca Dask-ML http://dask-ml.readthedocs.io/en/latest/ 

* ¿Cómo se comparan los tiempos de ejecución de tu búsqueda con la de Dask ML?
---  

En este caso para comparar ambas ejecuciones (SKLEARN vs DASK) seleccioné la Regresión Lineal:
+ Sklearn: **Ridge**
+ Dask: **LinearRegression**  
ya que tienen parámetros similares para probar la búsqueda de hiperparámetros.


Anteriormente intenté utilizar el _XGBRegressor_ pero el importarlo en Dask me arrojaba error de "No module named dask_xgboost"

### Importo librerías de SKLEARN

In [1]:
from timeit import default_timer as tic
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

---  
### Carga y limpieza de datos
#### Cargo datos - Muestro head() y descrbibe()

In [2]:
trips_df = pd.read_csv("/data/trips.csv")
trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00


In [3]:
trips_df.describe()

,fare_amount,passenger_count,taxi_id,tip_amount,trip_distance
count,9198.000000,9198.000000,9198.000000,9198.000000,9198.000000
mean,11.864122,1.688845,250.233746,1.540099,2.737848
std,10.339877,1.341433,144.640465,2.180261,3.296340
min,-3.500000,0.000000,1.000000,-0.700000,0.000000
25%,6.500000,1.000000,125.000000,0.000000,1.000000
50%,9.000000,1.000000,249.000000,1.050000,1.660000
75%,13.000000,2.000000,376.000000,2.100000,3.000000
max,230.000000,9.000000,499.000000,40.000000,41.940000


#### Calculo proporcion de propina como: "tip_amount"/"fare_amount"
#### Cambio car_type a numerico: 
+ car_type A = 0   
+ car_type B = 1  

In [4]:
# Calculo la proporción ("tip_prop") como ("tip_amount"/"fare_amount"). 
trips_df = trips_df[trips_df.fare_amount > 0]  # Elimino los 5 registros con fare_amount = 0
trips_df["tip_prop"] = trips_df.apply(lambda df: round(df["tip_amount"] / df["fare_amount"],1), axis=1)

# Cambio car_type a numerico: A: 0; B: 1.
trips_df.car_type = trips_df.apply(lambda df: 0 if df["car_type"] == 'A' else 1, axis=1)

trips_df.head(10)

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,tip_prop
0,0,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,0.2
1,0,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,0.0
2,0,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,0.1
3,0,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,0.1
4,0,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,0.2
5,0,9.5,5,1,1.50,2015-01-10 22:02:13,2015-01-10 21:51:09,1.71,0.2
6,0,8.0,1,1,1.50,2015-01-15 09:14:21,2015-01-15 09:04:01,1.27,0.2
7,0,7.5,4,1,1.60,2015-01-16 00:39:12,2015-01-16 00:31:11,1.55,0.2
8,0,6.0,5,1,1.56,2015-01-20 19:05:28,2015-01-20 18:58:23,0.54,0.3
9,0,52.0,1,1,30.00,2015-01-20 20:31:05,2015-01-20 19:59:55,15.38,0.6


#### Proceso datos de fechas:
+ Calculo la duracion del viaje y la convierto a minutos.  
+ Caclulo factor de interacción entre distancia y tiempo del viaje (trip_distance / length_trip):

In [5]:
# Convierto campos a formato Datetime:
trips_df.tpep_dropoff_datetime = pd.to_datetime(trips_df.tpep_dropoff_datetime)
trips_df.tpep_pickup_datetime = pd.to_datetime(trips_df.tpep_pickup_datetime)

# Funcion para convertir hora a minutos:
def convert_to_minutes(time):
    return (time.hour * 60 + time.minute)

# Convierto la duración del viaje a minutos:
trips_df["trip_time"] =  (pd.to_datetime(trips_df.tpep_dropoff_datetime - trips_df.tpep_pickup_datetime)
                            .apply(convert_to_minutes))

# Agrego factor de interacción entre distancia y tiempo del viaje (trip_distance / length_trip):
trips_df["rel_dist_time"] = trips_df.apply(lambda df: 0 if df["trip_time"] == 0 
                                                        else round(df["trip_distance"] / df["trip_time"],1), axis=1)


# Visualizo los datos ordenados en forma descendete por length_trip:
trips_df.sort_values(by=['trip_time'], ascending=0).head(10)

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,tip_prop,trip_time,rel_dist_time
4834,0,13.5,1,263,1.45,2015-01-14 18:36:39,2015-01-13 18:37:53,2.73,0.1,1438,0.0
667,1,5.0,1,38,0.00,2015-01-05 14:31:46,2015-01-04 14:44:09,0.53,0.0,1427,0.0
1892,1,6.0,5,102,0.34,2015-01-25 00:00:00,2015-01-24 09:27:33,1.29,0.1,872,0.0
1335,0,115.5,1,73,10.00,2015-01-26 16:37:03,2015-01-26 14:19:25,25.42,0.1,137,0.2
4125,0,92.8,1,222,22.00,2015-01-26 15:56:35,2015-01-26 13:55:40,21.40,0.2,120,0.2
2467,1,52.0,2,133,7.00,2015-01-26 15:22:12,2015-01-26 13:33:51,16.80,0.1,108,0.2
3541,0,52.0,1,192,11.60,2015-01-15 17:41:40,2015-01-15 16:15:09,17.50,0.2,86,0.2
3657,0,52.0,1,198,10.00,2015-01-15 17:12:21,2015-01-15 15:47:43,17.20,0.2,84,0.2
3089,0,52.0,1,167,6.00,2015-01-08 08:57:10,2015-01-08 07:33:37,17.40,0.1,83,0.2
5745,0,50.0,1,314,11.20,2015-01-15 17:08:39,2015-01-15 15:47:41,8.80,0.2,80,0.1


In [6]:
# Deje solo los registros con una diracion menor a 800
# Solo hay 3 que superan esta duracion y son:
# .-trip_distance: 2.73 - length_trip: 1438 min.
# .-trip_distance: 0.53 - length_trip: 1427 min.
# .-trip_distance: 1.29 - length_trip:  872 min
                            
trips_df = trips_df[trips_df.trip_time < 800]

---  
## SKLEARN
#### Separo datos en Entrenamiento y Pruaba:

In [7]:
# Features:
columns = ["car_type", "fare_amount", "passenger_count", "trip_distance", "trip_time", "rel_dist_time"]

X_train, X_test, y_train, y_test = train_test_split(
    trips_df[columns],     # Features
    trips_df["tip_prop"],  # Target
    test_size=0.3)

#### Defino parametros a probar:

In [8]:
# Construct some pipelines
pipe_lr = Pipeline([('scl', StandardScaler()),
                    ('lr', Ridge())])

# Parametros a probar:
grid_params = [{'lr__alpha': [2, 5, 10],
                'lr__tol': [1e-4, 1e-3, 1e-2]}]

# Armo grid search:
jobs = -1
grid = GridSearchCV(estimator=pipe_lr,
            param_grid=grid_params,
            cv=10, 
            n_jobs=jobs)

# Lista de modelos a correr:
# En este caso se esta probando solo 1 modelo pero podría extenderse a mas.
grids = [grid]
models = ['LinearRegression - Ridge']

#### Defino función para ejecutar el modelo:

In [9]:
def fit_sklearn(grids, models, X_train_p, y_train_p):
    best_acc = 1
    best_gs = ''        
    
    for idx, gs in enumerate(grids):
        print('Ajustando modelo (SKLEARN): ' + models[idx])

        # Fit model:
        gs.fit(X_train_p, y_train_p)
        gs.predict(X_train_p)    
        
        # Score:
        score = gs.score(X_train_p, y_train_p)
        print('Score: %.4f' % score + "\n")
        
        # Best model:
        if score < best_acc:
            best_acc = score
            best_gs = gs
    
    return(best_gs)

### sklearn - Secuencial
#### Ejecuto función y calculo el tiempo

In [10]:
%%time
lr_model_sklearn = fit_sklearn(grids, models, X_train, y_train)

Ajustando modelo (SKLEARN): LinearRegression - Ridge
Score: 0.0004

CPU times: user 150 ms, sys: 50 ms, total: 200 ms
Wall time: 914 ms


---
### Importo librerías de DASK

In [11]:
from timeit import default_timer as tic
from dask_ml.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from dask import persist, compute
from dask_ml.model_selection import GridSearchCV
import pandas as pd

from dask_ml.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import dask.dataframe as dd

#### Covierto dataframe Pandas a Dask

In [12]:
trips_dask = dd.from_pandas(trips_df, npartitions=1)

#### Separo datos en Entrenamiento y Pruaba:

In [13]:
df_train, df_test = trips_dask.random_split([0.7, 0.3], random_state=2)

columns = ["car_type", "fare_amount", "passenger_count", "trip_distance", "trip_time", "rel_dist_time"]

# Features y Target
X_train, y_train = df_train[columns], df_train['tip_prop'] 
X_test, y_test = df_test[columns], df_test['tip_prop']

X_train = X_train.values.compute()
y_train = y_train.values.compute()

#### Defino parametros a probar:

In [14]:
# Construct some pipelines
pipe_lr = Pipeline([('scl', StandardScaler()),
                    ('lr', LinearRegression())])

# Parametros a probar:
grid_params = [{'lr__C': [2, 5, 10],
                'lr__tol': [1e-4, 1e-3, 1e-2]}]

# Armo grid search:
jobs = -1
grid = GridSearchCV(estimator=pipe_lr,
            param_grid=grid_params,
            cv=10, 
            n_jobs=jobs)

# Lista de modelos a correr:
# En este caso se esta probando solo 1 modelo pero podría extenderse a mas.
grids = [grid]
models = ['LinearRegression']

#### Defino función para ejecutar el modelo:

In [15]:
def fit_dask(grids, models, X_train_p, y_train_p):
    best_acc = 1
    best_gs = ''        
    
    for idx, gs in enumerate(grids):
        print('Ajustando modelo (DASK): ' + models[idx])

        # Fit model:
        gs.fit(X_train_p, y_train_p)
        gs.predict(X_train_p)    
        
        # Score:
        score = gs.score(X_train_p, y_train_p)
        print('Score: %.4f' % score + "\n")
        
        # Best model:
        if score < best_acc:
            best_acc = score
            best_gs = gs
    
    return(best_gs)

### DASK - Secuencial
#### Ejecuto función y calculo el tiempo

In [16]:
%%time
lr_model_dask = fit_dask(grids, models, X_train, y_train)

Ajustando modelo (DASK): LinearRegression
Score: 0.1561

CPU times: user 3.02 s, sys: 4.68 s, total: 7.7 s
Wall time: 4.22 s


#### Best params

In [17]:
lr_model_dask.best_params_

{'lr__C': 2, 'lr__tol': 0.01}

### DASK - Distribuido
#### Ejecuto función y calculo el tiempo

In [18]:
from dask.distributed import Client
client = Client("scheduler:8786")

In [19]:
%%time
lr_model_distr = fit_dask(grids, models, X_train, y_train)

Ajustando modelo (DASK): LinearRegression
Score: 0.2733

CPU times: user 280 ms, sys: 50 ms, total: 330 ms
Wall time: 31 s


In [20]:
lr_model_distr.best_params_

{'lr__C': 10, 'lr__tol': 0.01}

### Respuesta:
En este caso, para la cantidad de datos a procesar (9198 registros), el costo de ejecución en paralelo es superior al 
costo de ejecución secuancial.   
El tiempo de ejecución de Sklaern fue considerablemente menor a Dask.

---  
**Bonus**

Haz lo mismo utilizando Spark ML

* ¿Cómo se comparan los tiempos de ejecución de Spark vs Dask?

Usa los datos en s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv

* ¿Cambia alguno de los resultados anteriores?